# Generators use opmath

In [1]:
import pennylane as qml

## Story's description

More context: https://xanaduhq.slack.com/archives/C06DX049G49/p1705355151489519

Some operators have a generator method and we want to make sure that they are using the new opmath (in particular, no Tensor or Hamiltonian instances)

AC:

no generator method uses hardcoded Hamiltonian or Tensor instances

### I copy and paste here the *Slack* description of the thread in which the story was born

**Christina Lee**
I think we also need to switch all the generators to `Prod` as well.

**Korbinian Kottmann**
Can you elaborate, what do you mean by switching the generators? (edited) 

**Korbinian Kottmann**
Like switch definitions like these to use s_prod, prod, and sum instead of *, @ and + for construction?
https://github.com/PennyLaneAI/pennylane/blob/master/pennylane/ops/qubit/qchem_ops.py#L316
qchem_ops.py
    def generator(self):
<https://github.com/PennyLaneAI/pennylane|PennyLaneAI/pennylane>PennyLaneAI/pennylane | Added by GitHub

**Christina Lee**
The generators might be hard coded to tensor or hamiltonian.

**Korbinian Kottmann**
Do you have an example? Would the example above constitute as such by having the generator defined as scalar * (op1 @ op2) ?
qchem_ops.py
    def generator(self):
<https://github.com/PennyLaneAI/pennylane|PennyLaneAI/pennylane>PennyLaneAI/pennylane | Added by GitHub

**Christina Lee**
`DoubleExcitation` is returning a Hamiltonian.

**Korbinian Kottmann**
Thanks!

**Korbinian Kottmann**
Created a story so we dont forget about this (n.d.r.: this one)

## Primi pensieri

Penso che vogliano che io controlli che i `generators` (qualunque cosa essi siano) degli `operators` stiano usando le funzioni del modulo `pennylane/ops/opmath`, visto che Tensor e Hamiltonian sono in fase di deprecazione.

Alcune domande:

- Come faccio a capire quali `generators` devo guardare?
- Come faccio a sapere se il `generator` che sto guardando deve essere modificato? La TZ parlava di `DoubleExcitation` (qualunque cosa sia), quindi potrei partire from there

### Deeper description:

Many operators have a generator method defined, e.g.
https://github.com/PennyLaneAI/pennylane/blob/master/pennylane/ops/qubit/qchem_ops.py#L598

The task of this story is to make sure that these returns new opmath instances `(SProd, Prod, Sum)`. In particular, currently the default for python operators `@` , `+` and `*` is to use "old opmath" and creates `Tensor` and `Hamiltonian` instances.

So the task is basically to change in all `Operator.generator` definitions the following:

`x * op` -> `qml.s_prod(x, op)` , `x+y` -> `qml.sum(x, y)` , `x @ y` -> `qml.prod(x, y)` and `qml.Hamiltonian(coeffs, ops)` -> `qml.dot(coeffs, ops)` 

then everything should be (nested) instances of `SProd` , `Prod` and `Sum` - the new opmath classes.

I'd approach the task as follows:
repo-search (e.g. in visual code with cmd-shift-f) `def generator(` in `pennylane/pennylane/` and make the above changes.

Check if tests fail/pass.

If fail if the test just needs an update (e.g. changing and `isinstance(op, Hamiltonian)` to `isinstance(op, Sum)`)

If there is no failing test to change, consider updating the test of the generator method to check `isinstance(op, Sum)` (or whichever class the result is). Ideally if there already a test for the generator method in the class, update this one. If it doesnt fit, add a new one

- Changed `/ops/qubit/parametric_ops_multi_qubit.py`, line 776 

    from:

    ```
    def generator(self):
        return -0.5 * PauliX(wires=self.wires[0]) @ PauliX(wires=self.wires[1])
    ```    
    to:

    ```
    def generator(self):
        return qml.s_prod(-0.5, qml.prod(PauliX(wires=self.wires[0]), PauliX(wires=self.wires[1])))
    ```    

- Stessa modifica alla linea 912           

- Stessa modifica alla linea 1055         

- Changed line 1238 

    from :

    ```
    def generator(self):
        return 0.25 * (
            PauliX(wires=self.wires[0]) @ PauliX(wires=self.wires[1])
            + PauliY(wires=self.wires[0]) @ PauliY(wires=self.wires[1])
        )

    ```

    to :

    ```
    def generator(self):
        return qml.s_prod(
            0.25,
            qml.sum(
                qml.prod(PauliX(wires=self.wires[0]), PauliX(wires=self.wires[1])),
                qml.prod(PauliY(wires=self.wires[0]), PauliY(wires=self.wires[1])),
            ),
        )
    ```

STOP: devo interrompere questo reportage altrimenti ci metto troppo       

In [2]:
op = qml.CRX(0.6, wires=[0, 1])
qml.generator(op)
#    (Projector([1], wires=[0]) @ X(1), -0.5)

(X(1) @ Projector(array([1]), wires=[0]), -0.5)

In [3]:
type(qml.s_prod(-0.5, qml.prod(qml.PauliX(wires=[1]), qml.PauliX(wires=[0]))))

pennylane.ops.op_math.sprod.SProd

In [4]:
import functools
from operator import matmul

old = -0.5 * functools.reduce(matmul, [qml.PauliZ(w) for w in [0, 1, 2]])
print(type(old))
print(old)

<class 'pennylane.ops.qubit.hamiltonian.Hamiltonian'>
  (-0.5) [Z0 Z1 Z2]


In [5]:
new = qml.s_prod(-0.5, functools.reduce(matmul, [qml.PauliZ(w) for w in [0, 1, 2]]))
print(type(new))
print(new)

<class 'pennylane.ops.op_math.sprod.SProd'>
-0.5 * (Z(0) @ Z(1) @ Z(2))


In [6]:
type(old)

pennylane.ops.qubit.hamiltonian.Hamiltonian

In [7]:
old.compare

<bound method Hamiltonian.compare of <Hamiltonian: terms=1, wires=[0, 1, 2]>>

In [8]:
from pennylane.qchem.tapering import (
    _kernel,
    _reduced_row_echelon,
    clifford,
    optimal_sector,
    taper_hf,
    taper_operation,
    _split_pauli_sentence,
    _taper_pauli_sentence,
)

from pennylane import numpy as np

"""
 @pytest.mark.parametrize(
    ("operation", "op_wires", "op_gen"),
    [
        (
            lambda phi, wires: qml.QubitUnitary(
                qml.math.array(
                    [
                        [qml.math.cos(phi / 2), 0, 0, -1j * qml.math.sin(phi / 2)],
                        [0, qml.math.cos(phi / 2), -1j * qml.math.sin(phi / 2), 0],
                        [0, -1j * qml.math.sin(phi / 2), qml.math.cos(phi / 2), 0],
                        [-1j * qml.math.sin(phi / 2), 0, 0, qml.math.cos(phi / 2)],
                    ]
                ),
                wires=wires,
            ),
            [0, 2],
            lambda phi, wires: qml.Hamiltonian(
                [-0.5 * phi], [qml.PauliX(wires=wires[0]) @ qml.PauliX(wires=wires[1])]
            ),
        ),
    ],
)
"""
phi = 0.1

operation = qml.math.cos(phi / 2), 0, 0, -1j * qml.math.sin(phi / 2)

op_wires = ([0, 2],)


def test_taper_matrix_ops(op_wires=[0, 2]):
    """Test that taper_operation can be used with gate operation built using matrices"""

    op_wires = [0, 2]

    symbols, geometry, charge = (
        ["He", "H"],
        np.array([[0.0, 0.0, 0.0], [0.0, 0.0, 1.4588684632]]),
        1,
    )
    mol = qml.qchem.Molecule(symbols, geometry, charge)
    hamiltonian = qml.qchem.diff_hamiltonian(mol)(geometry)

    generators = qml.symmetry_generators(hamiltonian)
    paulixops = qml.paulix_ops(generators, len(hamiltonian.wires))
    paulix_sector = optimal_sector(hamiltonian, generators, mol.n_electrons)
    wire_order = hamiltonian.wires

    taper_op1 = taper_operation(
        qml.IsingXX,
        generators,
        paulixops,
        paulix_sector,
        wire_order,
        op_wires=op_wires,
    )
    assert callable(taper_op1)

In [9]:
# test_taper_matrix_ops()

In [23]:
from pennylane.ops.op_math.pow import Pow

"""Test that the generator is the base's generator multiplied by the power."""
z = 2.5
base = qml.RX(2.34, wires=0)
op = Pow(base, z)

base_gen_op, base_gen_coeff = qml.generator(base, format="prefactor")
op_gen_op, op_gen_coeff = qml.generator(op, format="prefactor")

assert qml.math.allclose(base_gen_coeff * z, op_gen_coeff)
assert base_gen_op.__class__ is op_gen_op.__class__

In [26]:
type(base_gen_coeff * z)

numpy.float64

In [11]:
type(op_gen_op)

pennylane.ops.qubit.non_parametric_ops.X

In [12]:
type(op_gen_coeff)

numpy.float64

In [13]:
type(base_gen_op)

pennylane.ops.qubit.non_parametric_ops.X

In [14]:
type(base_gen_coeff)

numpy.float64

In [30]:
qml.generator(base, format="arithmetic")

-0.5 * X(0)

In [31]:
qml.generator(op, format="arithmetic")

-1.25 * X(0)

In [27]:
qml.s_prod(base_gen_coeff, z)

-0.5 * 2.5

In [28]:
op_gen_coeff

-1.25

In [32]:
op_gen_op, op_gen_coeff = qml.generator(op, format="prefactor")

## This is just a matter of style, ma ci sono priorities ben more urgent!

In [19]:
wires = [0, 1]

w1, w2 = wires

In [20]:
w0, w1, w2, w3 = [0, 1, 2, 3]
coeffs = [0.0625, 0.0625, -0.0625, 0.0625, -0.0625, 0.0625, -0.0625, -0.0625]
obs = [
    qml.X(w0) @ qml.X(w1) @ qml.X(w2) @ qml.Y(w3),
    qml.X(w0) @ qml.X(w1) @ qml.Y(w2) @ qml.X(w3),
    qml.X(w0) @ qml.Y(w1) @ qml.X(w2) @ qml.X(w3),
    qml.X(w0) @ qml.Y(w1) @ qml.Y(w2) @ qml.Y(w3),
    qml.Y(w0) @ qml.X(w1) @ qml.X(w2) @ qml.X(w3),
    qml.Y(w0) @ qml.X(w1) @ qml.Y(w2) @ qml.Y(w3),
    qml.Y(w0) @ qml.Y(w1) @ qml.X(w2) @ qml.Y(w3),
    qml.Y(w0) @ qml.Y(w1) @ qml.Y(w2) @ qml.X(w3),
]
qml.Hamiltonian(coeffs, obs)

  (-0.0625) [X0 Y1 X2 X3]
+ (-0.0625) [Y0 X1 X2 X3]
+ (-0.0625) [Y0 Y1 X2 Y3]
+ (-0.0625) [Y0 Y1 Y2 X3]
+ (0.0625) [X0 X1 X2 Y3]
+ (0.0625) [X0 X1 Y2 X3]
+ (0.0625) [X0 Y1 Y2 Y3]
+ (0.0625) [Y0 X1 Y2 Y3]


In [21]:
qml.sum(
    qml.s_prod(0.0625, qml.X(w0) @ qml.X(w1) @ qml.X(w2) @ qml.Y(w3)),
    qml.s_prod(0.0625, qml.X(w0) @ qml.X(w1) @ qml.Y(w2) @ qml.X(w3)),
    qml.s_prod(-0.0625, qml.X(w0) @ qml.Y(w1) @ qml.X(w2) @ qml.X(w3)),
    qml.s_prod(0.0625, qml.X(w0) @ qml.Y(w1) @ qml.Y(w2) @ qml.Y(w3)),
    qml.s_prod(-0.0625, qml.Y(w0) @ qml.X(w1) @ qml.X(w2) @ qml.X(w3)),
    qml.s_prod(0.0625, qml.Y(w0) @ qml.X(w1) @ qml.Y(w2) @ qml.Y(w3)),
    qml.s_prod(-0.0625, qml.Y(w0) @ qml.Y(w1) @ qml.X(w2) @ qml.Y(w3)),
    qml.s_prod(-0.0625, qml.Y(w0) @ qml.Y(w1) @ qml.Y(w2) @ qml.X(w3)),
)

(
    0.0625 * (X(0) @ X(1) @ X(2) @ Y(3))
  + 0.0625 * (X(0) @ X(1) @ Y(2) @ X(3))
  + -0.0625 * (X(0) @ Y(1) @ X(2) @ X(3))
  + 0.0625 * (X(0) @ Y(1) @ Y(2) @ Y(3))
  + -0.0625 * (Y(0) @ X(1) @ X(2) @ X(3))
  + 0.0625 * (Y(0) @ X(1) @ Y(2) @ Y(3))
  + -0.0625 * (Y(0) @ Y(1) @ X(2) @ Y(3))
  + -0.0625 * (Y(0) @ Y(1) @ Y(2) @ X(3))
)

In [22]:
coeffs = [0.0625, 0.0625, -0.0625, 0.0625, -0.0625, 0.0625, -0.0625, -0.0625]
obs = [
    qml.X(w0) @ qml.X(w1) @ qml.X(w2) @ qml.Y(w3),
    qml.X(w0) @ qml.X(w1) @ qml.Y(w2) @ qml.X(w3),
    qml.X(w0) @ qml.Y(w1) @ qml.X(w2) @ qml.X(w3),
    qml.X(w0) @ qml.Y(w1) @ qml.Y(w2) @ qml.Y(w3),
    qml.Y(w0) @ qml.X(w1) @ qml.X(w2) @ qml.X(w3),
    qml.Y(w0) @ qml.X(w1) @ qml.Y(w2) @ qml.Y(w3),
    qml.Y(w0) @ qml.Y(w1) @ qml.X(w2) @ qml.Y(w3),
    qml.Y(w0) @ qml.Y(w1) @ qml.Y(w2) @ qml.X(w3),
]

op = qml.s_prod(scalar=coeffs, operator=obs)